## Lab 7: Eel Distribution Modeling with XGBoost

Author: Haylee Oyler

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint

# Download the datasets
# model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
# eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")

# Working locally because it's faster...
model_data = pd.read_csv("data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("data/eval.data.csv")

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [2]:
# Assign features
X = model_data.drop('Angaus', axis=1)
y = model_data['Angaus']

# Encode categorical variable with LE
le = LabelEncoder()
X['Method'] = le.fit_transform(X['Method'])

# Split into training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=808)

### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [3]:
# Initialize XGB model
xgb = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1, 
    early_stopping_rounds=50, 
    eval_metric="logloss", 
    random_state=808)

# Fit model
xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)])


[0]	validation_0-logloss:0.51354
[1]	validation_0-logloss:0.48803
[2]	validation_0-logloss:0.46738
[3]	validation_0-logloss:0.45126
[4]	validation_0-logloss:0.43766
[5]	validation_0-logloss:0.42555
[6]	validation_0-logloss:0.41728
[7]	validation_0-logloss:0.40769
[8]	validation_0-logloss:0.40082
[9]	validation_0-logloss:0.39562
[10]	validation_0-logloss:0.39263
[11]	validation_0-logloss:0.38984
[12]	validation_0-logloss:0.38489
[13]	validation_0-logloss:0.38327
[14]	validation_0-logloss:0.37803
[15]	validation_0-logloss:0.37652
[16]	validation_0-logloss:0.37337
[17]	validation_0-logloss:0.37156
[18]	validation_0-logloss:0.36987
[19]	validation_0-logloss:0.36868
[20]	validation_0-logloss:0.36809
[21]	validation_0-logloss:0.36721
[22]	validation_0-logloss:0.36763
[23]	validation_0-logloss:0.36570
[24]	validation_0-logloss:0.36577
[25]	validation_0-logloss:0.36666
[26]	validation_0-logloss:0.36559
[27]	validation_0-logloss:0.36552
[28]	validation_0-logloss:0.36422
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=50,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=1000,
              n_jobs=None, num_parallel_tree=None, random_state=808, ...)

In [4]:
# Print best number of trees
best_ntrees = xgb.best_iteration
print(f"Best number of trees {best_ntrees}")

Best number of trees 29


### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [5]:
# Initialize second XGB model
xgb2 = XGBClassifier(
    n_estimators=best_ntrees,
    learning_rate=0.1, 
    early_stopping_rounds=50, 
    eval_metric="logloss", 
    random_state=808)

# Fit model
xgb2.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Create a parameter distribution for learning rate
param_dist = {
    "learning_rate": uniform(0.01, 0.3), 
}

# Set up RandomizedSearchCV
rs = RandomizedSearchCV(
    xgb2, param_dist, n_iter=20, scoring='accuracy', 
    cv=3, verbose=False, random_state=808
)

# Run random search
rs.fit(X_train, y_train, eval_set=[(X_val, y_val)])

[0]	validation_0-logloss:0.51354
[1]	validation_0-logloss:0.48803
[2]	validation_0-logloss:0.46738
[3]	validation_0-logloss:0.45126
[4]	validation_0-logloss:0.43766
[5]	validation_0-logloss:0.42555
[6]	validation_0-logloss:0.41728
[7]	validation_0-logloss:0.40769
[8]	validation_0-logloss:0.40082
[9]	validation_0-logloss:0.39562
[10]	validation_0-logloss:0.39263
[11]	validation_0-logloss:0.38984
[12]	validation_0-logloss:0.38489
[13]	validation_0-logloss:0.38327
[14]	validation_0-logloss:0.37803
[15]	validation_0-logloss:0.37652
[16]	validation_0-logloss:0.37337
[17]	validation_0-logloss:0.37156
[18]	validation_0-logloss:0.36987
[19]	validation_0-logloss:0.36868
[20]	validation_0-logloss:0.36809
[21]	validation_0-logloss:0.36721
[22]	validation_0-logloss:0.36763
[23]	validation_0-logloss:0.36570
[24]	validation_0-logloss:0.36577
[25]	validation_0-logloss:0.36666
[26]	validation_0-logloss:0.36559
[27]	validation_0-logloss:0.36552
[28]	validation_0-logloss:0.36422
[0]	validation_0-logloss

RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=50,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_r...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None, n_estimators=29,
                                           n_jobs=None, num_parallel_tree=None,
                                           random_state=808, ...),
                   n_iter=20,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x17fa67a00>},
                   random_state=808, scoring='accuracy', verbose=False)

In [26]:
# Print best number of learners
best_lr = rs.best_params_['learning_rate']
print(f"Best learning rate: {best_lr:.4f}")

Best learning rate: 0.1561


### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [7]:
# Initialize third XGB model
xgb3 = XGBClassifier(
    n_estimators = best_ntrees,
    learning_rate = best_lr, 
    early_stopping_rounds=50, 
    eval_metric="logloss", 
    random_state=808)

# Fit model
xgb3.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Second param dist
param_dist2 = {
    "max_depth": randint(3, 10), 
    "min_child_weight": randint(1, 10),
    "gamma": uniform(0.05, 0.05)
}

# Set up RandomizedSearchCV
rs2 = RandomizedSearchCV(
    xgb3, param_dist2, 
    n_iter=20, scoring='accuracy', 
    cv=3, verbose=False, random_state=808
)

# Run random search
rs2.fit(X_train, y_train, eval_set=[(X_val, y_val)])


[0]	validation_0-logloss:0.49740
[1]	validation_0-logloss:0.46517
[2]	validation_0-logloss:0.44162
[3]	validation_0-logloss:0.42470
[4]	validation_0-logloss:0.41305
[5]	validation_0-logloss:0.39805
[6]	validation_0-logloss:0.38969
[7]	validation_0-logloss:0.38508
[8]	validation_0-logloss:0.38266
[9]	validation_0-logloss:0.37894
[10]	validation_0-logloss:0.37521
[11]	validation_0-logloss:0.37435
[12]	validation_0-logloss:0.37361
[13]	validation_0-logloss:0.37088
[14]	validation_0-logloss:0.36893
[15]	validation_0-logloss:0.36716
[16]	validation_0-logloss:0.36596
[17]	validation_0-logloss:0.36431
[18]	validation_0-logloss:0.36233
[19]	validation_0-logloss:0.36445
[20]	validation_0-logloss:0.36460
[21]	validation_0-logloss:0.36307
[22]	validation_0-logloss:0.36590
[23]	validation_0-logloss:0.36729
[24]	validation_0-logloss:0.37149
[25]	validation_0-logloss:0.37507
[26]	validation_0-logloss:0.37586
[27]	validation_0-logloss:0.37487
[28]	validation_0-logloss:0.37357
[0]	validation_0-logloss

RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=50,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_r...
                                           random_state=808, ...),
                   n_iter=20,
                   param_distributions={'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x17fa67a90>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x11199d0f0>,
                                        'min_child_weight': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x17fab3850>},
                   random_state=808, scoring='accuracy', verbose=False)

In [ ]:
# Print best tree parameters
best_tree_params = rs2.best_params_
print(f"Best tree parameters: {best_tree_params}")

Best tree parameters: {'gamma': 0.07888873060786751, 'max_depth': 6, 'min_child_weight': 9}


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and .10 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and .10
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [ ]:
# Initialize fourth XGB model
xgb4 = XGBClassifier(
    n_estimators=best_ntrees,
    learning_rate=best_lr,
    **best_tree_params, 
    early_stopping_rounds=50, 
    eval_metric="logloss", 
    random_state=808)

# Fit model
xgb4.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Third param dist
param_dist3 = {
    "subsample": uniform(0.5, 0.5),
    "colsample_bytree": uniform(0.5, 0.5) 
}

# Set up RandomizedSearchCV
rs3 = RandomizedSearchCV(
    xgb4, param_dist3, n_iter=20, scoring='accuracy', 
    cv=3, verbose=False, random_state=808
)

# Run random search
rs3.fit(X_train, y_train, eval_set=[(X_val, y_val)])

[0]	validation_0-logloss:0.50496
[1]	validation_0-logloss:0.47653
[2]	validation_0-logloss:0.45815
[3]	validation_0-logloss:0.44338
[4]	validation_0-logloss:0.43010
[5]	validation_0-logloss:0.41679
[6]	validation_0-logloss:0.40787
[7]	validation_0-logloss:0.39469
[8]	validation_0-logloss:0.38683
[9]	validation_0-logloss:0.38396
[10]	validation_0-logloss:0.37479
[11]	validation_0-logloss:0.37488
[12]	validation_0-logloss:0.37182
[13]	validation_0-logloss:0.36748
[14]	validation_0-logloss:0.36163
[15]	validation_0-logloss:0.36051
[16]	validation_0-logloss:0.36092
[17]	validation_0-logloss:0.36032
[18]	validation_0-logloss:0.35681
[19]	validation_0-logloss:0.36002
[20]	validation_0-logloss:0.35935
[21]	validation_0-logloss:0.35928
[22]	validation_0-logloss:0.35962
[23]	validation_0-logloss:0.35571
[24]	validation_0-logloss:0.35382
[25]	validation_0-logloss:0.35625
[26]	validation_0-logloss:0.35451
[27]	validation_0-logloss:0.35443
[28]	validation_0-logloss:0.35179
[0]	validation_0-logloss

RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=50,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           gamma=0.07888873060786751,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=...
                                           multi_strategy=None, n_estimators=29,
                                           n_jobs=None, num_parallel_tree=None,
                                           random_state=808, ...),
                   n_iter=20,
                   param_distributions={'colsample_bytree': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x17fab3400>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x180f2c310>},
                   random_state=808, scoring='accuracy', verbose=False)

In [ ]:
# Print best stochastic parameters
best_stochastic_params = rs3.best_params_
print(f"Best stochastic parameters: {best_stochastic_params}")

Best stochastic parameters: {'colsample_bytree': 0.8506612141420485, 'subsample': 0.7565024905664723}


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [ ]:
# Check column names for feature split
eval_data.columns

Index(['Angaus_obs', 'SegSumT', 'SegTSeas', 'SegLowFlow', 'DSDist',
       'DSMaxSlope', 'USAvgT', 'USRainDays', 'USSlope', 'USNative', 'DSDam',
       'Method', 'LocSed'],
      dtype='object')

In [17]:
# Assign features
X_eval= eval_data.drop('Angaus_obs', axis=1)
y_eval = eval_data['Angaus_obs']

# Encode categorical variable with LE
le = LabelEncoder()
X_eval['Method'] = le.fit_transform(X_eval['Method'])

# Initialize fifth XGB model
xgb5 = XGBClassifier(
    n_estimators=best_ntrees,
    learning_rate=best_lr,
    **best_tree_params, 
    **best_stochastic_params, 
    early_stopping_rounds=50, 
    eval_metric="logloss", 
    random_state=808)

# Fit model
xgb5.fit(X_train, y_train, eval_set=[(X_val, y_val)])
preds = xgb5.predict(X_eval)


[0]	validation_0-logloss:0.51462
[1]	validation_0-logloss:0.48384
[2]	validation_0-logloss:0.45974
[3]	validation_0-logloss:0.44225
[4]	validation_0-logloss:0.42726
[5]	validation_0-logloss:0.41540
[6]	validation_0-logloss:0.41097
[7]	validation_0-logloss:0.40217
[8]	validation_0-logloss:0.39609
[9]	validation_0-logloss:0.38965
[10]	validation_0-logloss:0.38358
[11]	validation_0-logloss:0.37791
[12]	validation_0-logloss:0.37704
[13]	validation_0-logloss:0.37737
[14]	validation_0-logloss:0.37486
[15]	validation_0-logloss:0.37272
[16]	validation_0-logloss:0.36932
[17]	validation_0-logloss:0.36870
[18]	validation_0-logloss:0.36931
[19]	validation_0-logloss:0.36964
[20]	validation_0-logloss:0.36782
[21]	validation_0-logloss:0.36833
[22]	validation_0-logloss:0.36640
[23]	validation_0-logloss:0.36580
[24]	validation_0-logloss:0.36140
[25]	validation_0-logloss:0.36009
[26]	validation_0-logloss:0.35680
[27]	validation_0-logloss:0.35686
[28]	validation_0-logloss:0.35660


### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [19]:
# Get probability predictions 
pred_probs = xgb5.predict_proba(X_eval)[:, 1]

# Calculate ROC score
auc_score = roc_auc_score(y_eval, pred_probs)
print(f"AUC Score: {auc_score:.4f}")


AUC Score: 0.8632


In [ ]:
# Get feature importance
feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': xgb5.feature_importances_})

# Sort by importance
feature_importance = feature_importance.sort_values(by="Importance", ascending=False)
feature_importance

# Visualize importance
# plt.figure(figsize=(10, 6))
# plt.barh(feature_importance['Feature'], feature_importance['Importance'], color='darkgreen')
# plt.xlabel("Feature Importance")
# plt.ylabel("Features")
# plt.title("XGBoost Feature Importance")
# plt.gca().invert_yaxis() 
# plt.show()

,Feature,Importance
0,SegSumT,0.242132
6,USRainDays,0.117139
10,Method,0.112598
8,USNative,0.100860
4,DSMaxSlope,0.088386
5,USAvgT,0.071412
7,USSlope,0.063946
11,LocSed,0.056243
3,DSDist,0.055992
1,SegTSeas,0.047643


### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

My model is fairly similar to that of Elith et al. We both got the same most important feature of `SegSumt` and third important feature of `Method`, but my second most important was `USRainDays` while Elith et al.'s was `USNative`. 

My AUC was very similar, with a score of 0.863 compared to Elith et al.'s 0.858.

WHAT'S THE OTHER WAY TO COMPARE MODELS??